# Plotting script

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as mpl
import math
import csv
import os
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from graph import Graph
from graph_helper import *
from functions import *
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'pdf.fonttype': 42})

## Helper functions

In [ ]:
# Create dataframe from csv files of different algorithms.
def create_dataframe(prefix:str, var_name:str, vars:list, algos:dict, round:int, mode="performance"):
    # Declare the data frame.
    df = pd.DataFrame(columns=['Algorithm', var_name])

    # Read data into dataframe.
    for var in vars:
        for algo in algos.keys():
            dir = "{}/{}={}/".format(prefix, var_name, var)
            file = "{}/{}-{}.csv".format(dir, algo, mode)
            if os.path.isfile(file):
                value = pd.read_csv(file, delimiter=",", header=None)
                df = pd.concat([df, value.iloc[round].T*100], ignore_index=True)
                df.loc[len(df)-len(value.iloc[round].T):,'Algorithm'] = algos[algo]
                df.loc[len(df)-len(value.iloc[round].T):, var_name] = var
    df = df.rename(columns={0: 'IRC (%)'})
    return df

# Create evolution dataframe from csv files of different algorithms.
def create_steps_dataframe(prefix:str, budgets, algos:dict):
    # Declare the data frame.
    df = pd.DataFrame(columns=['Algorithm', 'Budget'])
    # Read data into dataframe.
    for algo in algos.keys():
        file = "{}/{}-performance.csv".format(prefix, algo)
        if os.path.isfile(file):
            value = pd.read_csv("{}/{}-performance.csv".format(prefix, algo), delimiter=",", header=None)
            for budget in budgets:
                df = pd.concat([df, value.iloc[budget].T*100], ignore_index=True)
                df.loc[len(df)-len(value.iloc[budget].T):,'Algorithm'] = algos[algo]
                df.loc[len(df)-len(value.iloc[budget].T):, 'Budget'] = budget+1
    df = df.rename(columns={0: 'IRC (%)'})
    return df

def create_win_df(prefix:str, var_name:str, vars:list, algos:dict, round:int):
    df = create_dataframe(prefix, var_name, vars, algos, round)
    df = df.rename(columns={'Algorithm': 'Window Size'})
    stats = df.groupby([var_name, 'Window Size'])['IRC (%)'].agg(['mean', 'std'])
    df = pd.pivot_table(df, values="IRC (%)", index='Window Size', columns=var_name)
    return df, stats

##  Simulation results graphs

### Planning Time & Communication Probability

In [ ]:
# Set up the figure.
f, axes = plt.subplots(2, 3, figsize=(30, 20))
sns.set_theme(context='paper', style='whitegrid', palette='deep', font='Times New Roman', font_scale=2.5, color_codes=True, rc=None)
round = -1
planning = ['100', '250', '500', '750', '1000']
budgets = np.array(range(0,10))
algos = {"SW": "SW-MCTS", "Discounted": "Dec-MCTS", "Discounted-Reset": "Dec-MCTS-Res", "Central": "Central-MCTS"}

# Create dataframe static.
df_planning_sc = create_dataframe("../Data/Static/", "planning", planning, algos, round)
dup_dec = df_planning_sc[df_planning_sc['Algorithm'] == 'Dec-MCTS'].copy()
dup_dec.loc[:, 'Algorithm'] = 'Dec-MCTS-Reset'
df_planning_sc = pd.concat([df_planning_sc, dup_dec], ignore_index=True)

df_static = create_steps_dataframe("../Data/Static/Planning=500/", budgets, algos)
dup_dec = df_static[df_static['Algorithm'] == 'Dec-MCTS'].copy()
dup_dec.loc[:, 'Algorithm'] = 'Dec-MCTS-Reset'
df_static = pd.concat([df_static, dup_dec], ignore_index=True)
df_static['Budget'] = df_static['Budget']*10

# Create dataframe dynamic.
df_planning_dc = create_dataframe("../Data/DC/", "planning", planning, algos, round)
df_planning_lc = create_dataframe("../Data/LC/", "planning", planning, algos, round)

df_LC = create_steps_dataframe("../Data/LC/Planning=500/", budgets, algos)
df_LC['Budget'] = df_LC['Budget']*10
df_DC = create_steps_dataframe("../Data/DC/Planning=500/", budgets, algos)
df_DC['Budget'] = df_DC['Budget']*10

# Plot the average score with 95% confidence interval.
ax_1 = sns.lineplot(data=df_planning_sc, x="planning", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'g', 'skyblue'], markers=['v', 's', 'o', 'd'], ax=axes[0, 0])
ax_1.set(xlabel="Planning Time (iterations) \n (a)")

ax_2 = sns.lineplot(data=df_planning_lc, x="planning", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue', 'g'], markers=['v', 's', 'd', 'o'], ax=axes[0, 1])
ax_2.set(xlabel="Planning Time (iterations) \n (b)")

ax_3 = sns.lineplot(data=df_planning_dc, x="planning", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue', 'g'], markers=['v', 's', 'd', 'o'], ax=axes[0, 2])
ax_3.set(xlabel="Planning Time (iterations) \n (c)")

ax_4 = sns.lineplot(data=df_static, x='Budget', y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'g', 'skyblue'], markers=['v', 's', 'o', 'd'], ax=axes[1, 0])
ax_4.set(xlabel="% Travel Budget Used\n (d)")

ax_5 = sns.lineplot(data=df_LC, x='Budget', y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue', 'g'], markers=['v', 's', 'd', 'o'], ax=axes[1, 1])
ax_5.set(xlabel="% Travel Budget Used\n (e)")

ax_6 = sns.lineplot(data=df_DC, x='Budget', y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue', 'g'], markers=['v', 's', 'd', 'o'], ax=axes[1, 2])
ax_6.set(xlabel="% Travel Budget Used\n (f)")

for i in range(2):
    for j in range(3):
        axes[i, j].autoscale(enable=True, axis='x', tight=True)
        if i == 0:
            axes[i, j].set_ylim([30, 90]) 
        elif i ==1:
            axes[i, j].set_ylim([0, 90])
            axes[i, j].set_xticks([i for i in range(10,110,10)])
        axes[i, j].get_legend().remove()
handles, labels = axes[0, 1].get_legend_handles_labels()
leg = f.legend(handles, labels, markerscale=3, fontsize=25, loc='upper center', bbox_to_anchor=(0.5, 0.93), ncols=4)
for line in leg.get_lines():
    line.set_linewidth(5.0)

# Save figure to pdf.
# plt.savefig("../Figure/planning.svg", format="svg", bbox_inches="tight")

In [ ]:
# Set up the figure.
f, axes = plt.subplots(1, 3, figsize=(30, 10))
sns.set_theme(context='paper', style='whitegrid', palette='deep', font='Times New Roman', font_scale=2.5, color_codes=True, rc=None)
round = -1
comm = [0, 0.05, 0.1, 0.5, 1]
algos = {"SW": "SW-MCTS", "Discounted": "Dec-MCTS", "Discounted-Reset": "Dec-MCTS-Res"}

# Create dataframe static.
df_comm_sc = create_dataframe("../Data/Static/", "comm", comm, algos, round)
df_comm_sc['comm'] = df_comm_sc['comm']*100
dup_dec = df_comm_sc[df_comm_sc['Algorithm'] == 'Dec-MCTS'].copy()
dup_dec.loc[:, 'Algorithm'] = 'Dec-MCTS-Reset'
df_comm_sc = pd.concat([df_comm_sc, dup_dec], ignore_index=True)
df_comm_sc['IRC (%)'] = df_comm_sc['IRC (%)'] - df_planning_sc['IRC (%)'].where((df_planning_sc['Algorithm'] == 'Central-MCTS') & (df_planning_sc['planning'] == '500')).mean()

# Create dataframe dynamic.
df_comm_dc = create_dataframe("../Data/DC/", "comm", comm, algos, round)
df_comm_dc['comm'] = df_comm_dc['comm']*100
df_comm_dc['IRC (%)'] = df_comm_dc['IRC (%)'] - df_planning_dc['IRC (%)'].where((df_planning_dc['Algorithm'] == 'Central-MCTS') & (df_planning_dc['planning'] == '500')).mean()
df_comm_lc = create_dataframe("../Data/LC/", "comm", comm, algos, round)
df_comm_lc['comm'] = df_comm_lc['comm']*100
df_comm_lc['IRC (%)'] = df_comm_lc['IRC (%)'] - df_planning_lc['IRC (%)'].where((df_planning_lc['Algorithm'] == 'Central-MCTS') & (df_planning_lc['planning'] == '500')).mean()

# Plot the average score with 95% confidence interval.
ax_7 = sns.lineplot(data=df_comm_sc, x="comm", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue'], markers=['v', 's', 'd'], ax=axes[0])
ax_7.set(xlabel="Successful Communication Probability (%) \n (a)")

ax_8 = sns.lineplot(data=df_comm_lc, x="comm", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue'], markers=['v', 's', 'd'], ax=axes[1])
ax_8.set(xlabel="Successful Communication Probability (%) \n (b)")

ax_9 = sns.lineplot(data=df_comm_dc, x="comm", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue'], markers=['v', 's', 'd'], ax=axes[2])
ax_9.set(xlabel="Successful Communication Probability (%) \n (c)")

for i in range(3):
    axes[i].autoscale(enable=True, axis='x', tight=True)
    axes[i].set_ylim([-10, 40])
    axes[i].set(ylabel="% IRC Improvement Over Central-MCTS")
    axes[i].set_xticks([i*100 for i in comm])
    axes[i].get_legend().remove()
handles, labels = axes[2].get_legend_handles_labels()
leg = f.legend(handles, labels, markerscale=3, fontsize=25, loc='upper center', bbox_to_anchor=(0.5, 0.98), ncols=3)
for line in leg.get_lines():
    line.set_linewidth(5.0)

# Save figure to pdf.
# plt.savefig("../Figure/comm.svg", format="svg", bbox_inches="tight")

### Number of Budgets, Agents & Rewards

In [ ]:
# Set up the figure.
f, axes = plt.subplots(3, 3, figsize=(30, 30))
sns.set_theme(context='paper', style='whitegrid', palette='deep', font='Times New Roman', font_scale=2.5, color_codes=True, rc=None)
round = -1
action = ['6', '8', '10', '12', '14']
agent = ['6', '8', '10', '12', '14']
reward = ['100', '200', '300', '400', '500']
algos = {"SW": "SW-MCTS", "Discounted": "Dec-MCTS", "Discounted-Reset": "Dec-MCTS-Res", "Central": "Central-MCTS"}

# Create dataframe static.
df_action_sc = create_dataframe("../Data/Static", "action", action, algos, round)
dup_dec = df_action_sc[df_action_sc['Algorithm'] == 'Dec-MCTS'].copy()
dup_dec.loc[:, 'Algorithm'] = 'Dec-MCTS-Reset'
df_action_sc = pd.concat([df_action_sc, dup_dec], ignore_index=True)

df_agent_sc = create_dataframe("../Data/Static", "agent", agent, algos, round)
dup_dec = df_agent_sc[df_agent_sc['Algorithm'] == 'Dec-MCTS'].copy()
dup_dec.loc[:, 'Algorithm'] = 'Dec-MCTS-Reset'
df_agent_sc = pd.concat([df_agent_sc, dup_dec], ignore_index=True)

df_reward_sc = create_dataframe("../Data/Static", "reward", reward, algos, round)
dup_dec = df_reward_sc[df_reward_sc['Algorithm'] == 'Dec-MCTS'].copy()
dup_dec.loc[:, 'Algorithm'] = 'Dec-MCTS-Reset'
df_reward_sc = pd.concat([df_reward_sc, dup_dec], ignore_index=True)

# Create datafram dynamic.
df_action_lc = create_dataframe("../Data/LC", "action", action, algos, round)
df_agent_lc = create_dataframe("../Data/LC", "agent", agent, algos, round)
df_reward_lc = create_dataframe("../Data/LC", "reward", reward, algos, round)
df_action_dc = create_dataframe("../Data/DC", "action", action, algos, round)
df_agent_dc = create_dataframe("../Data/DC", "agent", agent, algos, round)
df_reward_dc = create_dataframe("../Data/DC", "reward", reward, algos, round)

# Plot the average action score with 95% confidence interval.
ax_1 = sns.lineplot(data=df_action_sc, x="action", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'g', 'skyblue'], markers=['v', 's', 'o', 'd'], ax=axes[0, 0])
ax_1.set(xlabel="Travel Budget (actions) \n (a)")

ax_2 = sns.lineplot(data=df_action_lc, x="action", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue', 'g'], markers=['v', 's', 'd', 'o'], ax=axes[0, 1])
ax_2.set(xlabel="Travel Budget (actions) \n (b)")

ax_3 = sns.lineplot(data=df_action_dc, x="action", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue', 'g'], markers=['v', 's', 'd', 'o'], ax=axes[0, 2])
ax_3.set(xlabel="Travel Budget (actions) \n (c)")

ax_4 = sns.lineplot(data=df_agent_sc, x="agent", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'g', 'skyblue'], markers=['v', 's', 'o', 'd'], ax=axes[1, 0])
ax_4.set(xlabel="Number of AUVs \n (d)")

ax_5 = sns.lineplot(data=df_agent_lc, x="agent", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue', 'g'], markers=['v', 's', 'd', 'o'], ax=axes[1, 1])
ax_5.set(xlabel="Number of AUVs \n (e)")

ax_6 = sns.lineplot(data=df_agent_dc, x="agent", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue', 'g'], markers=['v', 's', 'd', 'o'], ax=axes[1, 2])
ax_6.set(xlabel="Number of AUVs \n (f)")

ax_7 = sns.lineplot(data=df_reward_sc, x="reward", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'g', 'skyblue'], markers=['v', 's', 'o', 'd'], ax=axes[2, 0])
ax_7.set(xlabel="Number of Sensors \n (g)")

ax_8 = sns.lineplot(data=df_reward_lc, x="reward", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue', 'g'], markers=['v', 's', 'd', 'o'], ax=axes[2, 1])
ax_8.set(xlabel="Number of Sensors \n (h)")

ax_9 = sns.lineplot(data=df_reward_dc, x="reward", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=['r', 'b', 'skyblue', 'g'], markers=['v', 's', 'd', 'o'], ax=axes[2, 2])
ax_9.set(xlabel="Number of Sensors \n (i)")

for i in range(3):
    for j in range(3):
        # axes[i, j].legend(markerscale=3, fontsize=25, loc='best')
        axes[i, j].autoscale(enable=True, axis='x', tight=True)
        axes[i, j].set_ylim([20, 100])
        axes[i, j].get_legend().remove()
        handles, labels = axes[i, j].get_legend_handles_labels()
leg = f.legend(handles, labels, markerscale=3, fontsize=25, loc='upper center', bbox_to_anchor=(0.5, 0.92), ncols=4)
for line in leg.get_lines():
    line.set_linewidth(5.0)

# Save figure to pdf.
# plt.savefig("../Figure/action_agent_reward.svg", format="svg", bbox_inches="tight")

### Number of Changes & Changes Intensity

In [ ]:
# Set up the figure.
f, axes = plt.subplots(1, 4, figsize=(40, 10))
sns.set_theme(context='paper', style='whitegrid', palette='deep', font='Times New Roman', font_scale=2.5, color_codes=True, rc=None)
round = -1
rate = [0.2, 0.4, 0.6, 0.8, 1]
cable = [0.025, 0.05, 0.075, 0.1, 0.125]
percent = [0.2, 0.4, 0.6, 0.8, 1]
algos = {"SW": "SW-MCTS", "Discounted": "Dec-MCTS", "Discounted-Reset": "Dec-MCTS-Res", "Central": "Central-MCTS"}
palette=['r', 'b', 'skyblue', 'g']
markers=['v', 's', 'd', 'o']

# Create datafram number of changes.
df_rate_lc = create_dataframe("../Data/LC", "rate", rate, algos, round)
df_rate_lc['rate'] = df_rate_lc['rate']*10
df_rate_dc = create_dataframe("../Data/DC", "rate", rate, algos, round)
df_rate_dc['rate'] = df_rate_dc['rate']*10

# Create datafram changes intensity.
df_intensity_lc = create_dataframe("../Data/LC", "cable", cable, algos, round)
df_intensity_lc['cable'] = df_intensity_lc['cable']*1000
df_intensity_dc = create_dataframe("../Data/DC", "percent", percent, algos, round)
df_intensity_dc['percent'] = df_intensity_dc['percent']*200

# Plot the average score with 95% confidence interval.
ax_3 = sns.lineplot(data=df_intensity_lc, x="cable", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=palette, markers=markers, ax=axes[0])
ax_3.set(xlabel="Cable Length (m) \n (a)")
ax_3.set_xticks([i*1000 for i in cable])

ax_1 = sns.lineplot(data=df_rate_lc, x="rate", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=palette, markers=markers, ax=axes[1])
ax_1.set(xlabel="Number of Changes \n (b)")
ax_1.set_xticks([i*10 for i in rate])

ax_4 = sns.lineplot(data=df_intensity_dc, x="percent", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=palette, markers=markers, ax=axes[2])
ax_4.set(xlabel="Number of Changed Sensors \n (c)")
ax_4.set_xticks([i*200 for i in percent])

ax_2 = sns.lineplot(data=df_rate_dc, x="rate", y='IRC (%)', hue='Algorithm', style='Algorithm', errorbar='ci', linewidth=3, markersize=20, palette=palette, markers=markers, ax=axes[3])
ax_2.set(xlabel="Number of Changes \n (d)")
ax_2.set_xticks([i*10 for i in rate])

for i in range(4):
    # axes[i].legend(markerscale=3, fontsize=25, loc='best')
    axes[i].autoscale(enable=True, axis='x', tight=True)
    axes[i].set_ylim([30, 90])
    axes[i].get_legend().remove()
    handles, labels = axes[i].get_legend_handles_labels()
leg = f.legend(handles, labels, markerscale=4, fontsize=30, loc='upper center', bbox_to_anchor=(0.5, 1), ncols=4)
for line in leg.get_lines():
    line.set_linewidth(5.0)

# Save figure to pdf.
# plt.savefig("../Figure/frequency_intensity.svg", format="svg", bbox_inches="tight")

### Impact of window size with different planning time, number of budgets and agents

In [ ]:
# Set up the figure.
f, axes = plt.subplots(3, 3, figsize=(30, 30))
sns.set_theme(context='paper', style='whitegrid', palette='deep', font='Times New Roman', font_scale=2.5, color_codes=True, rc=None)
round = -1
planning = [100, 250, 500, 750, 1000]
action = [6, 8, 10, 12, 14]
agent = [6, 8, 10, 12, 14]
algos = {"SW-100": "100", "SW-300": "300", "SW-500": "500", "SW-750": "750", "SW-1000": "1000"}
stats = dict()
df = dict()
ax = dict()

# Create dataframe static.
df['planning_sc'], stats['planning_sc'] = create_win_df("../Data/Win_Static", "planning", planning, algos, round)
df['action_sc'], stats['action_sc'] = create_win_df("../Data/Win_Static", "action", action, algos, round)
df['agent_sc'], stats['agent_sc'] = create_win_df("../Data/Win_Static", "agent", agent, algos, round)

# Create datafram dynamic.
df['planning_lc'], stats['planning_lc'] = create_win_df("../Data/Win_LC", "planning", planning, algos, round)
df['action_lc'], stats['action_lc'] = create_win_df("../Data/Win_LC", "action", action, algos, round)
df['agent_lc'], stats['agent_lc'] = create_win_df("../Data/Win_LC", "agent", agent, algos, round)
df['planning_dc'], stats['planning_dc'] = create_win_df("../Data/Win_DC", "planning", planning, algos, round)
df['action_dc'], stats['action_dc'] = create_win_df("../Data/Win_DC", "action", action, algos, round)
df['agent_dc'], stats['agent_dc'] = create_win_df("../Data/Win_DC", "agent", agent, algos, round)

# Plot the average action score with 95% confidence interval.
ax['planning_sc'] = sns.heatmap(data=df['planning_sc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=planning, yticklabels=[algos[i] for i in algos.keys()], ax=axes[0, 0])
ax['planning_sc'].set(xlabel="Planning Time (iterations) \n (a)", ylabel="Window Size")

ax['planning_lc'] = sns.heatmap(data=df['planning_lc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=planning, yticklabels=[algos[i] for i in algos.keys()], ax=axes[0, 1])
ax['planning_lc'].set(xlabel="Planning Time (iterations) \n (b)", ylabel="Window Size")

ax['planning_dc'] = sns.heatmap(data=df['planning_dc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=planning, yticklabels=[algos[i] for i in algos.keys()], ax=axes[0, 2])
ax['planning_dc'].set(xlabel="Planning Time (iterations) \n (c)", ylabel="Window Size")

ax['action_sc'] = sns.heatmap(data=df['action_sc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=action, yticklabels=[algos[i] for i in algos.keys()], ax=axes[1, 0])
ax['action_sc'].set(xlabel="Travel Budget (actions) \n (d)", ylabel="Window Size")

ax['action_lc'] = sns.heatmap(data=df['action_lc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=action, yticklabels=[algos[i] for i in algos.keys()], ax=axes[1, 1])
ax['action_lc'].set(xlabel="Travel Budget (actions) \n (e)", ylabel="Window Size")

ax['action_dc'] = sns.heatmap(data=df['action_dc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=action, yticklabels=[algos[i] for i in algos.keys()], ax=axes[1, 2])
ax['action_dc'].set(xlabel="Travel Budget (actions) \n (f)", ylabel="Window Size")

ax['agent_sc'] = sns.heatmap(data=df['agent_sc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=agent, yticklabels=[algos[i] for i in algos.keys()], ax=axes[2, 0])
ax['agent_sc'].set(xlabel="Number of AUVs \n (g)", ylabel="Window Size")

ax['agent_lc'] = sns.heatmap(data=df['agent_lc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=agent, yticklabels=[algos[i] for i in algos.keys()], ax=axes[2, 1])
ax['agent_lc'].set(xlabel="Number of AUVs \n (h)", ylabel="Window Size")

ax['agent_dc'] = sns.heatmap(data=df['agent_dc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=agent, yticklabels=[algos[i] for i in algos.keys()], ax=axes[2, 2])
ax['agent_dc'].set(xlabel="Number of AUVs \n (i)", ylabel="Window Size")

for var in df.keys():
    ax[var].invert_yaxis()
    column_max = df[var].idxmax(axis=0)
    for col, variable in enumerate(df[var].columns):
        position = df[var].index.get_loc(column_max[variable])
        ax[var].add_patch(Rectangle((col, position),1,1, fill=False, edgecolor='red', lw=3))

for key in stats.keys():
    ci95 = []

    for i in stats[key].index:
        _, s = stats[key].loc[i]
        ci95.append(1.96*s/math.sqrt(20))

    stats[key]['ci'] = ci95
    stats[key] = stats[key].drop(columns='std')
    # stats[key].to_csv("../Data/Win/df_win_{}.csv".format(key))
ci_mean = 0
for key in stats.keys():
    ci_mean += stats[key]['ci'].mean()
print("Mean confidence interval: ", ci_mean/4)

# Save figure to pdf.
# plt.savefig("../Figure/action_agent_reward.svg", format="svg", bbox_inches="tight")

### Impact of different window size with different number of changes and changes intensity

In [ ]:
# Set up the figure.
f, axes = plt.subplots(1, 4, figsize=(40, 10))
sns.set_theme(context='paper', style='whitegrid', palette='deep', font='Times New Roman', font_scale=2.5, color_codes=True, rc=None)
round = -1
rate = [0.2, 0.4, 0.6, 0.8, 1]
cable = [0.025, 0.05, 0.075, 0.1, 0.125]
percent = [0.2, 0.4, 0.6, 0.8, 1]
algos = {"SW-100": "100", "SW-300": "300", "SW-500": "500", "SW-750": "750", "SW-1000": "1000"}
stats = dict()
df = dict()
ax = dict()

# Create datafram number of changes.
df['rate_lc'], stats['rate_lc'] = create_win_df("../Data/Win_LC", "rate", rate, algos, round)
df['rate_dc'], stats['rate_dc']  = create_win_df("../Data/Win_DC", "rate", rate, algos, round)

# Create datafram changes intensity.
df['intensity_lc'], stats['intensity_lc'] = create_win_df("../Data/Win_LC", "cable", cable, algos, round)
df['intensity_dc'], stats['intensity_dc'] = create_win_df("../Data/Win_DC", "percent", percent, algos, round)

# Plot the average score with 95% confidence interval.
ax['rate_lc'] = sns.heatmap(data=df['rate_lc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=[int(i*10) for i in rate], yticklabels=[algos[i] for i in algos.keys()], ax=axes[0])
ax['rate_lc'].set(xlabel="Number of Changes \n (a)", ylabel="Window Size")

ax['intensity_lc'] = sns.heatmap(data=df['intensity_lc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=[int(i*1000) for i in cable], yticklabels=[algos[i] for i in algos.keys()], ax=axes[1])
ax['intensity_lc'].set(xlabel="Cable Length (m) \n (b)", ylabel="Window Size")

ax['rate_dc'] = sns.heatmap(data=df['rate_dc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=[int(i*10) for i in rate], yticklabels=[algos[i] for i in algos.keys()], ax=axes[2])
ax['rate_dc'].set(xlabel="Number of Changes \n (c)", ylabel="Window Size")

ax['intensity_dc'] = sns.heatmap(data=df['intensity_dc'], annot=True, fmt=".2f", cbar=False, cmap='Blues', xticklabels=[int(i*100) for i in percent], yticklabels=[algos[i] for i in algos.keys()], ax=axes[3])
ax['intensity_dc'].set(xlabel="% Change \n (d)", ylabel="Window Size")

for var in df.keys():
    ax[var].invert_yaxis()
    column_max = df[var].idxmax(axis=0)
    for col, variable in enumerate(df[var].columns):
        position = df[var].index.get_loc(column_max[variable])
        ax[var].add_patch(Rectangle((col, position),1,1, fill=False, edgecolor='red', lw=3))

for key in stats.keys():
    ci95 = []

    for i in stats[key].index:
        _, s = stats[key].loc[i]
        ci95.append(1.96*s/math.sqrt(20))

    stats[key]['ci'] = ci95
    stats[key] = stats[key].drop(columns='std')
    # stats[key].to_csv("../Data/Win/df_win_{}.csv".format(key))
ci_mean = 0
for key in stats.keys():
    ci_mean += stats[key]['ci'].mean()
print("Mean confidence interval: ", ci_mean/4)

# Save figure to pdf.
# plt.savefig("../Figure/frequency_intensity.svg", format="svg", bbox_inches="tight")

## Rollout paths of different algorithms

### Generate the graph

In [ ]:
# Declare some parameters.
xL = -2     # min of the x-axis
xH = 2      # max of the x-axis
yL = -2     # min of the y-axis
yH = 2      # max of the y-axis
obsMask = np.array([
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        ])                                  # the obstacles mask.

# Set up for simulation.
reward_radius = 0.05                              # reward disk radius.
n_agents = 10

# Import the graph.
config = 0
G = Graph(xL, xH, yL, yH, reward_radius, obsMask)
path = "../Data/Config_{}".format(config)
G, agents, rewards, nodes, _, n_nodes, n_rewards = import_graph(G, path)
G.add_reward(rewards)
print("Generating the graph environment with {} agents, {} rewards, and {} intermediate nodes".format(n_agents, n_rewards, n_nodes))

### Rollouts

In [ ]:
# Mapping values.
trial = 1
n_round = -1

# with open("../Data/LC/Planning=500//Discounted-rollout-C{}-T{}.csv".format(config, trial), 'r') as csv_file:
#     csv_reader = list(csv.reader(csv_file))
#     # N_iter = len(csv_reader)
#     # for current_iter in range(N_iter-1, N_iter):
#     discounted_joint_path = parse(csv_reader[n_round][0])

# with open("../Data/Static/Planning=500//SW-rollout-C{}-T{}.csv".format(config, trial), 'r') as csv_file:
#     csv_reader = list(csv.reader(csv_file))
#     # N_iter = len(csv_reader)
#     # for current_iter in range(N_iter-1, N_iter):
#     sw_joint_path = parse(csv_reader[n_round][0])

with open("../Data/Static/Planning=500//Central-rollout-C{}-T{}.csv".format(config, trial), 'r') as csv_file:
    csv_reader = list(csv.reader(csv_file))
    # N_iter = len(csv_reader)
    # for current_iter in range(N_iter-1, N_iter):
    central_joint_path = parse(csv_reader[n_round][0])

mpl.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(12,12))
plt.xticks([])
plt.yticks([])
plt.xlim(G.xL, G.xH)
plt.ylim(G.yL, G.yH)

G.draw_agents(agents)
# G.draw_edges()

for reward in rewards:
    circle = plt.Circle((reward[0], reward[1]), reward_radius, fc='none', ec='green', linewidth=3)
    plt.gca().add_patch(circle)

# for key in discounted_joint_path.keys():
#     for i in range(len(discounted_joint_path[key])):
#         x_index = []
#         y_index = []
#         if ~np.isnan(discounted_joint_path[key][i]):
#             start_node = G.edges_list[int(discounted_joint_path[key][i])][0]
#             end_node = G.edges_list[int(discounted_joint_path[key][i])][1]
#             start_node_coor = G.find_coordinates(start_node)
#             end_node_coor = G.find_coordinates(end_node)
#             x_index.extend([start_node_coor[0], end_node_coor[0]])
#             y_index.extend([start_node_coor[1], end_node_coor[1]])
#         plt.plot(x_index, y_index, 'blue', linewidth=4, solid_joinstyle='round', solid_capstyle='round')
#     dx = (end_node_coor[0] - start_node_coor[0])/100
#     dy = (end_node_coor[1] - start_node_coor[1])/100
#     plt.arrow(end_node_coor[0], end_node_coor[1], dx, dy, shape='full', lw=4, length_includes_head=True, head_width=.04, color='blue', label="Dec-MCTS" if key == 0 else "")

# for key in sw_joint_path.keys():
#     for i in range(len(sw_joint_path[key])):
#         x_index = []
#         y_index = []
#         if ~np.isnan(sw_joint_path[key][i]):
#             start_node = G.edges_list[int(sw_joint_path[key][i])][0]
#             end_node = G.edges_list[int(sw_joint_path[key][i])][1]
#             start_node_coor = G.find_coordinates(start_node)
#             end_node_coor = G.find_coordinates(end_node)
#             x_index.extend([start_node_coor[0], end_node_coor[0]])
#             y_index.extend([start_node_coor[1], end_node_coor[1]])
#         plt.plot(x_index, y_index, 'red', linewidth=4, solid_joinstyle='round', solid_capstyle='round')
#     dx = (end_node_coor[0] - start_node_coor[0])/100
#     dy = (end_node_coor[1] - start_node_coor[1])/100
#     plt.arrow(end_node_coor[0], end_node_coor[1], dx, dy, shape='full', lw=3, length_includes_head=True, head_width=.04, color='red', label="SW-MCTS" if key == 0 else "")

for key in central_joint_path.keys():
    for i in range(len(central_joint_path[key])):
        x_index = []
        y_index = []
        if ~np.isnan(central_joint_path[key][i]):
            start_node = G.edges_list[int(central_joint_path[key][i])][0]
            end_node = G.edges_list[int(central_joint_path[key][i])][1]
            start_node_coor = G.find_coordinates(start_node)
            end_node_coor = G.find_coordinates(end_node)
            x_index.extend([start_node_coor[0], end_node_coor[0]])
            y_index.extend([start_node_coor[1], end_node_coor[1]])
        plt.plot(x_index, y_index, 'gold', linewidth=4, solid_joinstyle='round', solid_capstyle='round')
    dx = (end_node_coor[0] - start_node_coor[0])/100
    dy = (end_node_coor[1] - start_node_coor[1])/100
    plt.arrow(end_node_coor[0], end_node_coor[1], dx, dy, shape='full', lw=3, length_includes_head=True, head_width=.04, color='gold', label="Cen-MCTS" if key == 0 else "")

plt.draw()
plt.legend(loc="upper left", fontsize=20, edgecolor="black", facecolor="whitesmoke", frameon=True)
# fig.savefig("../Figure/Static_Rollout.pdf")